In [1]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

timestr = time.strftime("%Y%m%d-%H%M%S")
df = pd.read_csv('data/example_pcap3.csv')

#dataset = pd.get_dummies(dataset)
#dataset = dataset.replace(np.nan, 0)
#dataset = dataset.to_numpy()

df = df[df["quic"] == "quic"] #only need quic packets bc these are the only packets for the video

df = df.reset_index(drop=True) #reset df indices after dropping rows

client = df['ip.src'][0] #client sends first request to server

df = df[df['ip.dst'] == client] #only look at packets incoming to the client

df = df.reset_index(drop=True) #reset df indices after dropping rows


#go through and calculate chunk sizes

chunk_size_data = []

chunk_size = 0
init = 1
prev_start_time = 0
for i in range(len(df)):
    if("tls" in df['frame.protocols']):
        continue
    else:
        if(prev_start_time == 0):
            prev_start_time = df['frame.time_relative'][i]
        if(df['frame.time_relative'][i]>prev_start_time + 1): #more than 0.3s since last packet, means new chunk
            chunk_size_data.append(chunk_size)
            chunk_size = 0
            prev_start_time = df['frame.time_relative'][i]
        chunk_size += int(df['frame.len'][i])
        prev_start_time = df['frame.time_relative'][i]

chunk_size_data=np.asarray(chunk_size_data)



#detect new chunk on special packet - not possible
'''
chunk_size_data = []

chunk_size = 0
init = 1
#prev_start_time = 0
for i in range(len(df)):
    if(df['Info'][i] != "Protected Payload (KP0)"):
        if(df['Info'][i].startswith("Initial")): #new chunk
            if(init == 1):
                continue
            chunk_size_data.append(chunk_size)
            chunk_size = 0
                
        continue
        df = df[df['Destination'] == client]
    if(df['Info'][i] == "Protected Payload (KP0)" and df['Destination'][i] == client):
        if(init == 1):
            init = 0
        chunk_size += int(df['Length'][i])

chunk_size_data=np.asarray(chunk_size_data)
print(chunk_size_data)
print(chunk_size_data.shape)
'''

'\nchunk_size_data = []\n\nchunk_size = 0\ninit = 1\n#prev_start_time = 0\nfor i in range(len(df)):\n    if(df[\'Info\'][i] != "Protected Payload (KP0)"):\n        if(df[\'Info\'][i].startswith("Initial")): #new chunk\n            if(init == 1):\n                continue\n            chunk_size_data.append(chunk_size)\n            chunk_size = 0\n                \n        continue\n        df = df[df[\'Destination\'] == client]\n    if(df[\'Info\'][i] == "Protected Payload (KP0)" and df[\'Destination\'][i] == client):\n        if(init == 1):\n            init = 0\n        chunk_size += int(df[\'Length\'][i])\n\nchunk_size_data=np.asarray(chunk_size_data)\nprint(chunk_size_data)\nprint(chunk_size_data.shape)\n'

In [2]:
i=0
while(i<chunk_size_data.shape[0]):
    if chunk_size_data[i]<5000:
        chunk_size_data = np.delete(chunk_size_data, [i], 0)
    else:
        i+=1
print(chunk_size_data)
print(chunk_size_data.shape)

[   8104   17716 2369474 4209494  371973    8315 1795187  399179 1798123
  383424  426071 1779323  303586  535608    8367 2521968 5285118  547180
 1906163    8579  325722  559155   83731 2316099  549036  149254  487696
  137801  165913  810134  144870  174192  156906  161390  140332  503352
  141924  148979  150593  151028  155004  647986  158236  165683  153215
  165459  157419  659753  135118  158572  149883  161162  150706  658131
  147038  151589  144830  156346  152609  472325  137380  120876   93466
   82610 1731651   33229 1524535  691156  845773 1565894  656325 1396023
  682034 1363941  668957 2188825  683748 1450717  665700 1394531  681915
 1357276  665677 1433052  653472  674455 1289235  664315 1329816  268728
   90409 2238447 2693610 3811710 1864588    8530 1879263  819000 1891722
 1885260  811890 1862502 1908547  806494 1721036 1753365 2570706 1886428
 2138658 1872353 1934777 1911581 2103866 1886392 2386405 1754571  141517
 2844649 3342825  512103    8273 1927897  479658 20